# EMTF++

## Table of Contents

- [Introduction](#Introduction)
  
  * [Version](#Version)
  * [Requirements](#Requirements)
  * [New forward muon detectors](#New-forward-muon-detectors)
  * [EMTF firmware, emulator, fastsim](#EMTF-firmware,-emulator,-fastsim)
  
- [Algorithm](#Algorithm)

  * [Primitive conversion](#Primitive-conversion)
  * [Pattern recognition](#Pattern-recognition)
  * [Track building](#Track-building)
  * [Parameter assignment](#Parameter-assignment)
  
- [Performance plots](#Performance-plots)

  * [Resolution](#Resolution)
  * [Efficiency](#Efficiency)
  * [Rates](#Rates)
  * [Robustness](#Robustness)

- [Future plans](#Future-plans)

  * Run 3 configuration
  * Displaced muons
  * Extension to overlap region
  * Etc

## Introduction

### Version

- __v1.1.0__ (2018-11-06): Fixed the incorrect number of bunches used in the rate calculation: ~~1866~~ 2808

- __v1.0.0__ (2018-10-18): Reworked the patterns and the NN to reduce resource usage, without affecting the performance. Used as the starting point for (i) CMSSW integration. (ii) Firmware implementation.
  * Regarding the trigger primitives, `CMSSW_10_1_5` + changes from `cms-l1t-offline:l1t-phase2-v2.16.6` is used. For rate studies, official NeutrinoGun PU200 sample (500k events) `/SingleNeutrino/PhaseIIFall17D-L1TPU200_93X_upgrade2023_realistic_v5-v1/GEN-SIM-DIGI-RAW` is used.

### Requirements

- <span style="font-size: 120%">Maintain sensitivity to electroweak scale physics at higher luminosity and pileup of the HL-LHC</span>
  * Report all standalone muon coordinates and momenta in convention to facilitate __global correlation__ with tracks from the __Track Trigger__
    + The tracker will have far better p<sub>T</sub> resolution for rate reduction
  * Incorporate additional HL-LHC forward muon detectors to improve 
    + Efficiency, redundancy, and improved standalone p<sub>T</sub> measurement
  * Maintain standalone muon trigger (without track combination) for sufficiently high p<sub>T</sub> threshold
    + HL-LHC is “only” 3-4X higher lumi, and we increased the max L1 rate
- <span style="font-size: 120%">Add sensitivity to new physics scenarios, i.e. acceptance to __displaced muons and HSCPs__ from long-lived particle decays</span>
  * Additional patterns/logic and momentum assignment required (vertex constrained and not)

### New forward muon detectors

<img src="figures/cms_upg_o_g_b_ni_gem_re1_me0_grid_160229.png" width="800px"/>
<a href="figures/cms_upg_o_g_b_ni_gem_re1_me0_grid_160229.pdf">Download PDF</a>

- New forward detectors improve redundancy, efficiency and timing (only iRPC)
  * GEM detectors: GE1/1, GE2/1
  * iRPC detectors: RE3/1, RE4/1
  * ME0 detector
- New detectors also provide additional inputs and improved angular information (bend angles) for better correlation with TT tracks and better standalone muon p<sub>T</sub> measurement
- This requires increasing the bandwidth to the L1 muon electronics and adding more algorithm logic capability

### EMTF firmware, emulator, fastsim

While EMTF is really one algorithm, it is implemented in __two__ different ways: (i) firmware, written in Verilog and implemented in the MTF7 board; (ii) emulator, written in C++ and implemented as part of CMSSW software ([GitHub repo](https://github.com/cms-sw/cmssw/tree/master/L1Trigger/L1TMuonEndCap)). In the perfect world, they should be identical, but this is not always the case, and it is very important to recognize this fact when referring to the details of the EMTF algorithm.

In this document, I will try my best to be clear about which algorithm is being discussed. I'll denote the firmware algorithm as simply <span style="color: red; font-weight: bold">"firmware"</span>, and the emulator algorithm as <span style="color: red; font-weight: bold">"emulator"</span>. Whenever there is discrepancy, __firmware is always right__. Alex has written a description of the firmware algorithm [here](figures/EMU_TF_algorithm-1.pdf). 

In addition, in the course of developing the Phase-2 EMTF algorithm (namely EMTF++), I made another implementation in Python, and can be used outside of CMSSW. I'll denote this as <span style="color: red; font-weight: bold">"fastsim"</span>. The main reason I do this is because (i) it's very tedious to work with the emulator directly, and (ii) it allows me to use certain powerful Python libraries directly <sup><a href="#myfootnote1">[1]</a></sup>. Basically, I have made simple ntuples and I can run Python scripts on them to do analysis much more quickly (instead of going through the full-blown CMSSW). It allows me to make experiments more quickly.

<a name="myfootnote1"><sup>1</sup></a>   I found the scientific stack in Python very powerful. These are the libraries I have used in my study: rootpy, NumPy, SciPy, Scikit-learn, Keras, Tensorflow.

At the time of writing, this is what I have:
- EMTF: firmware, emulator
- EMTF++: fastsim

My goal is to have the firmware and emulator versions of EMTF++ ready soon. (when?!)

## Algorithm

The EMTF system consists of 12 EMTF processors &mdash; 6 for each endcap, 1 for each 60&deg; trigger sector (at 15&deg;, 75&deg;, ...). Each sector works independently.

<img src="figures/trigger_sectors.png" width="500px"/>

To ensure coverage near the sector boundaries, each sector N also shares certain chambers with the neighbor sector N+1. So, although each sector is nominally 60&deg;, it actually covers 70&deg; in the rings with 10&deg;-wide chambers, or 80&deg; in the rings with 20&deg;-wide chambers.
- ME1: chambers with CSCID=3,6,9, subsector=2 are shared
- ME2,3,4: chambers with CSCID=3,9 are shared

EMTF is a successor of CSCTF. It was installed during Phase-1 upgrade (see [Phase-1 L1 Trigger Upgrade TDR](https://cds.cern.ch/record/1556311/) for more info).

The EMTF algorithm starts by receiving the hits, or trigger primitives, and ends by producing muon tracks with certain information: (most importantly) p<sub>T</sub> , &eta;, &#981;, track quality, and its associated hits. I will break the EMTF algorithm into 4 building blocks. 

<img src="figures/algorithm_blocks.gv.svg" width="800px"/>

Each block may consist of smaller blocks. EMTF++ is an extension of the EMTF algorithm, and it follows the same design. In the following, I'll try to describe how each block works and what are the differences between EMTF vs EMTF++.

### Primitive conversion

In Phase-1, EMTF receives two types of trigger primitives: <span style="color: #31bd38">&#11035;</span> __CSC__ and <span style="color: #0096ff">&#11035;</span> __RPC__ hits. In Phase-2, EMTF++ will receive additional <span style="color: #e63600">&#11035;</span> __GEM__, <span style="color: #9437ff">&#11035;</span> __iRPC__ and <span style="color: #ffa25f">&#11035;</span> __ME0__ trigger primitives. 

The most important function here is to convert strip (half-strip, actually) and wire numbers to integer &#981; and &theta; units. The integer &#981; unit is a local coordinate defined within a sector. The scale of the unit is 1/60 = 0.0167&deg;, internally called "1/8-strip", encoded as a 13-bit integer (0 to 8191). The position 0 corresponds to -22&deg; from the lower boundary of the sector (since Sep 2018). To convert from the integer &#981; unit to CMS global coordinate (in degrees):

\begin{align}
\phi_{\text{sector}} &= \phi_{\text{integer}} / 60 - 22 \\
\phi_{\text{CMS}} &= \phi_{\text{sector}} + 15 + (\text{sector}-1) \times 60 \\
& \quad \  \text{sector} \in \{1,2,3,4,5,6\}
\end{align}

The integer &theta; unit has a scale of approx 0.285&deg;, encoded as a 7-bit integer (0 to 127). The position 0 corresponds to 8.5&deg;. To convert from the integer &theta; unit to CMS global coordinate (in degrees):

\begin{align}
\theta_{\text{CMS}} &= \theta_{\text{integer}} \cdot \left(\frac{45-8.5}{128}\right) + 8.5\\
\end{align}

The conversions from half-strip number to the integer &#981; unit and from wire number to the integer &theta; unit are done using simple linear equations. The constants (i.e. offsets & slopes) are derived offline and stored in the Block RAM in the firmware. There are additional corrections to the &theta; conversion for ME1/1 chambers with tilted wires, which I won't describe here.

<span style="font-size: 120%; font-weight: bold">(i) CSC</span>

The CSC trigger primitive is called a Local Charge Track (LCT). It is a track segment reconstructed from up to 6 hits in the CSC detector. LCTs are built by the Trigger MotherBoard (TMB) and then sent to the Muon Port Card (MPC). The data are concentrated by MPC and then sent to EMTF.

CSC geometry is very interesting. When designing the trigger, one must pay attention to the following features:
- ME1 has 3 rings, and the chambers are 10&deg; wide. ME2,3,4 have 2 rings, but the ring 1 chambers are 20&deg; wide; and the ring 2 chambers are 10&deg; wide.
- ME1 ring 1 is located at a different z position (~6m) compared to rings 2 & 3 (~7m). The magnetic field is very different for ME1/1 because it is inside the solenoid ($B_z = 3.8{\text{T}}$). The ME1/1 wires are tilted by 29&deg; to compensate for the drift of the electrons in the strong magnetic field.
- ME1/1 is further split into ME1/1a and ME1/1b at &eta;~2.05. They have different numbers of strips.
  * ME1/1a has 48 strips, ME1/1b has 64 strips, ME1/2 has 80 strips, ME1/3 has 64 strips, ME2,3,4 have 80 strips
- ME1/3 is only 80% (?) efficient.
- Depending on the location of a chamber (even or odd chamber number, ME1,2 or ME3,4, positive or negative endcap), a chamber can be closer or further from the interaction point. We assign a F/R (front or rear) bit for each chamber to take this into account.

See detector note [IN-2007/024](figures/IN2007_024.pdf) for more details.

The strips and wires are independent of each other. If there are two LCTs in a given chamber, we will get 2 strip numbers and 2 wire numbers. We do not know which strip number and which wire number to be paired up. Thus, we consider all the possibilities. Since there are at most 2 LCTs in a given chamber, there are at most 4 combinations to consider.

A summary of the exact CSC inputs was given by Alex:
- &#981; of each LCT: 13-bit values
- &theta; of each LCT: 7-bit values
- pattern ID for each LCT: 4-bit values
- FR bit for each LCT: 1-bit values
- Pattern mask: 4-bit mask showing which CSC stations are present in track (ME1,2,3,4)

In 
<span style="display: inline-block; border-radius: .4rem; padding: 0.35rem .6rem; background: #3776ab; color: #ffffff; vertical-align: middle;">EMTF++</span>
, we will be using the same primitive conversion modules that are already in the firmware and in the emulator. In addition, we want to add a new __bend__ variable, if it can be done in the TMB. TAMU and UCLA have shown that doing a least squares fit to the CLCT comparator digis can improve the &#981; position (see Andrew Peck's [talk](https://indico.cern.ch/event/712513/contributions/2959915/attachments/1630667/2599369/20180410_otmb_pattern_logic.pdf)). But I find it more useful to extract the local bending angle from the fit, where the bend is defined as the &Delta;&#981; between the hit &#981;'s at the innermost and outermost layers of the CSC detector.

Basically, consider there are up to 6 hits in the 6 layers of the CSC detector. We can do a very simple linear fit with $x$ = layer number and $y$ = hit &#981;. Extract the slope of the fit and multiply it by 6 to get the new bend variable. The bend is a floating-point value. I multiply it by 4 (arbitrarily) before quantization, which should fit into 7 bits (-64 to 63). Since the CSC comparator digis are in half-strip unit, the bend is in 1/8-strip unit. It is only useful for ME1/1 and ME1/2. For more details, see slides 7-9 in my [talk](https://indico.cern.ch/event/759873/contributions/3151457/subcontributions/264210/attachments/1722105/2780661/2018-09-25_phase2_emtf_v1.pdf).

<span style="font-size: 120%; font-weight: bold">(ii) RPC</span>

In Phase-1, EMTF has received endcap RPC hit information to improve efficiency and redundancy. Specifically, the RPC chambers: RE1/2, RE2/2, RE3/2, RE3/3, RE4/2, and RE4/3 are received. To distinguish from the Phase 2 iRPC detectors, I will also refer to the Phase 1 RPC as "old RPC". 

Old RPC hits are pre-processed by CPPF (which stands for Concentration, PreProcessing and Fanout), which clusterizes the RPC digis and performs coordinate conversion, before sending up to 2 clusters per RPC chamber to the EMTF. The integer &#981; and &theta; units used for the CPPF converted hits are 4x coarser compared to the units used for CSC, because of the worse detector resolutions.

Old RPC geometry info:
- 10&deg;-wide chambers, 32 strips per chamber, 3 &eta; partitions per chamber (or "rolls").
- Max cluster width is 3 strips.
- To ensure coverage near the sector boundaries, each sector also receives one 10&deg; chamber from the neighbor sector.
- Because RPC is a single-layer detector, it is more prone to noise.

A summary of the exact CPPF inputs was given by Alex:
- &#981; of each hit: 11-bit values
- &theta; of each hit: 5-bit values
- Pattern mask: 4-bit mask showing which RPC stations are present in track (RE1,2,3,4)

No changes for 
<span style="display: inline-block; border-radius: .4rem; padding: 0.35rem .6rem; background: #3776ab; color: #ffffff; vertical-align: middle;">EMTF++</span>
. But I believe CPPF will be replaced by newer electronics during Phase-2 upgrade.

<span style="font-size: 120%; font-weight: bold">(iii) iRPC</span>

iRPC are improved RPC detectors that will be available for Phase-2. These are the iRPC chambers: RE3/1, RE4/1. In general, iRPC is very similar to RPC, but with better spatial resolutions and timing resolution (~1.5 ns).  See [Phase-2 Muon TDR](https://cds.cern.ch/record/2283189) for more info.

iRPC geometry info:
- 20&deg;-wide chambers, 192 strips per chamber, 5 &eta; partitions per chamber (or "rolls").
- Since the iRPC chamber is 2x larger than the old RPC chamber, with 6x more strips per chamber, the &#981; resolution is 3x better.
- Like RPC, iRPC is a single-layer detector, and it is more prone to noise.
- Note: I was told that iRPC will have 2D readout, which will provide better &eta; or &theta; resolution. However, the 2D readout was not present in the CMSSW release that I'm using. Also, I'm not sure about the exact cluster width cut. I apply a cut of 9 strips (arbitrarily). I also only keep up to 2 clusters per chamber. To the extent where it's possible, I try to treat RPC and iRPC hits the same, so that it doesn't complicate the workflow too much.

For the coordinate conversion, since iRPC has better resolution, I want to use the EMTF integer units, not the 4x coarser CPPF integer units. However, due to a bug that I only discovered just now, I was using the 4x coarser CPPF integer units. I will correct this in my results in the future.

Currently, I don't use the 1.5ns timing in the baseline simulation, because I'm wary that the simulation performance might not be very realistic. It can always be added in the future when we are more confident. In fact, it's supposed to be very important to help triggering for HSCPs (heavy stable charged particles).

<span style="font-size: 120%; font-weight: bold">(iv) GEM</span>

GEM (Gas Electron Multiplier) is a new type of detector to be installed at GE1/1 and GE2/1. In particular, GE1/1 can provide a bend angle between GE1/1 and ME1/1, which is claimed to significantly improve the p<sub>T</sub> measurement. GE1/1 is planned to be installed during LS2, and be ready for physics in Run 3; GE2/1 will be installed during LS3 together with the rest of new Phase-2 detectors. See [GEM TDR](https://cds.cern.ch/record/2021453) for more info about GE1/1, and [Phase-2 Muon TDR](https://cds.cern.ch/record/2283189) for more info about GE2/1.

GEM geometry info:
- GE1/1 is 10&deg; wide with 192 pads; GE2/1 is 20&deg; wide with 384 pads. A pad is 2 strips ganged together (double-strip), and it is the unit that will be transmitted to the EMTF. Same &#981; resolution for GE1/1 and GE2/1.
- Both GE1/1 and GE2/1 have 8 &eta; partitions per chamber.
- Both GE1/1 and GE2/1 are made of 2-layer detectors.

According to Sven Dildick (TAMU), the GEM pads will be clustered before they are sent to EMTF. The max cluster width is 8 pads, and there are at most 8 clusters per chamber. According to his instructions, EMTF is supposed to declusterize the clusters to retrieve the original pads and use them (up to 64 pads) in the trigger, so as not to "lose" resolution. I'm not sure what is the merit of these instructions, but I have followed the instructions in my study.

Furthermore, TAMU has developed algorithms that combine GEM & CSC hits to form a new ILT (integrated LCT) trigger primitives. Doing this allows us to (i) reduce the number-of-layers requirement for the CSC LCT from 4 to 3 which improves efficiency; (ii) improve background rejection with the increased number of hits during the trigger primitive reconstruction. The ILT algorithm is currently not used in my study, but it can always be added in the future. 

<span style="font-size: 120%; font-weight: bold">(v) ME0</span>

ME0 is a 6-layer GEM detector that will be mounted at the "nose" of the endcap. A ME0 segment will be built by dedicated electronics, and we will receive the &#981;, &theta; (or &eta;), the local bending angle and the quality. The bend is the &Delta;&#981; between the innermost and the outermost ME0 detector layers. See [Phase-2 Muon TDR](https://cds.cern.ch/record/2283189) for more info.

ME0 extends from &eta; = 2.0 to 2.8. Although this allows EMTF to trigger beyond &eta; of 2.4, I don't include it as part of the current results, because it adds extra rates, and makes it hard to do rate comparisons.

ME0 geometry info:
- ME0 chambers are 20&deg; wide, 384 strips (192 pads) per chamber, 8 &eta; partitions per chamber.
- I'm not sure we will receive ME0 pad numbers, or we will receive the converted &#981; and &eta; coordinates. I assume the latter currently.

ME0 has really excellent efficiency and background rejection. It works so well that I'm kind of worried that the simulation performance might not be realistic. So, I think it's important to add some safety margin even though we see a large rate reduction.

<span style="font-size: 120%; font-weight: bold">(vi) Summary</span>

I try to summarize all the spatial resolutions that we have in the following table. (Maybe adding DT too?)

<img src="figures/chamber_types_and_resolutions.png" width="800px"/>

The relative &#981; resolution is relative to CSC 10&deg; chambers (non-ME1/1). For ME0, I assume the &#981; coordinate is extracted from the fit, so I use 384 strips and a factor of 1/sqrt(6). Note that the exact number of bits used for the new Phase-2 muon detectors might not be simulated correctly at the moment.

In the Phase-2 scenario, each sector processor may receive up to 95 links. For the numbers of input links coming from each detector type, see slide 15 in Darin's [talk](https://indico.cern.ch/event/768406/contributions/3192943/subcontributions/266196/attachments/1743806/2822333/Phase2EndcapMuonTriggerDemos.pdf). 

### Pattern recognition

The trajectory of a muon as it passes through the endcap muon stations is bent by the magnetic field. Muons with low p<sub>T</sub> are bent more compared to muons with high p<sub>T</sub>. Note that as the magnetic field diminishes at large z, there is very little bending of the muon trajectories in the outer stations even for low p<sub>T</sub> muons. In EMTF, we use patterns of different straightness to detect muons by matching hits to the patterns. Each sector is divided into 4 "zones" (0-3) along the &theta; direction, and pattern matching is done in each zone. Multiple patterns may be activated ("fired") by the same hits, so a ghost busting step is done after pattern matching to get rid of the duplicates ("ghosts").

Due to latency in receiving the RPC hits from CPPF, RPC hits are not included in pattern matching. Due to possible BX misidentification, all the CSC hits from 2 consecutive BX's are used (since June 2018). The boundaries of the 4 zones are (0, 41, 49, 87, 127) in terms of the integer &theta; unit. To ensure coverage near the boundaries, an overlap of 2 units is allocated. E.g. CSC hits with &theta; from 0 to 43 are included in zone 0, and CSC hits with &theta; from 40 to 51 are included in zone 1, and so on.

The patterns that are used at the beginning of EMTF ("asymmetric") and the symmetric patterns that have been in use since Sep 2016 are shown below. In the asymmetric case, patterns of straightness 0-3 also have their mirrors, so there are 9 patterns in total. The unit used in pattern matching is a coarser &#981; unit, which is internally called "quad-strip". The scale of the quad-strip unit is approx 0.5&deg;, which is 32x coarser than the integer &#981; unit described previously (1/8-strip).

<div class="row" style="width:800px">
  <br/>
  <div class="column" style="width:48%; float: left">
    Asymmetric patterns<br/>
    <img src="figures/emtf_patterns.png" style="width:100%"/>
  </div>
  <div class="column" style="width:48%; float: left">
    Symmetric patterns<br/>
    <img src="figures/emtf_patterns_symmetric.png" style="width:100%"/>
  </div>
</div>

ME2 is called the key station. These patterns are repeated for 160 times for each quad-strip unit (from 0 to 159) at ME2 in order to fully cover the 80&deg; in &#981; in one sector (when including the chambers from the neighbor sector). Therefore, the total number of patterns in one sector is:

$$
4\ \text{(zone)} \times 5\ \text{(pattern)} \times 160\ \text{(quad-strip)} = 3,200
$$

In pattern ghost busting, each fired patterns ("road") is assigned a 6-bit quality code. Each road is compared against its neighbors (+/-1 quad-strip at the key station). If the quality code of a road is lower than either of the neighbors, the road is cancelled. The quality code is defined as follows:

<table style="width:500px">
<thead>
<tr>
<th style="width:14%; text-align: left;">Bit number</th>
<th style="width:14%; text-align: left;">bit 5</th>
<th style="width:14%; text-align: left;">bit 4</th>
<th style="width:14%; text-align: left;">bit 3</th>
<th style="width:14%; text-align: left;">bit 2</th>
<th style="width:14%; text-align: left;">bit 1</th>
<th style="width:14%; text-align: left;">bit 0</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align: left;">Contents</td>
<td style="text-align: center;">S2</td>
<td style="text-align: center;">ME1 hit</td>
<td style="text-align: center;">S1</td>
<td style="text-align: center;">ME2 hit</td>
<td style="text-align: center;">S0</td>
<td style="text-align: center;">ME3 or ME4 hit</td>
</tr>
</tbody>
</table>

where (S2, S1, S0) are (bit 2, bit 1, bit 0) of the pattern straightness.

I made a number of changes in the
<span style="display: inline-block; border-radius: .4rem; padding: 0.35rem .6rem; background: #3776ab; color: #ffffff; vertical-align: middle;">EMTF++</span>
algorithm, which I will describe below.

<span style="font-size: 120%; font-weight: bold">(i) Virtual station</span>

Although there are nominally 4 ME stations, ME1/1 and ME1/2 are very distinct due to very different magnetic field. The addition of ME0 also breaks the traditional four-station scheme. Thus, I decided to split the muon detectors by station and detector types into 12 virtual stations.

<table style="width:700px">
<tr>
<th style="width:7.6%; text-align: left; background-color: white">Station number</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #31bd38">s0</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #31bd38">s1</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #31bd38">s2</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #31bd38">s3</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #31bd38">s4</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #0096ff">s5</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #0096ff">s6</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #0096ff">s7</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #0096ff">s8</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #ff3300">s9</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #ff3300">s10</td>
<th style="width:7.6%; text-align: center; font-size: 120%; color: white; background-color: #ffa25f">s11</td>
</tr>
<tr>
<td style="text-align: left; background-color: white">Muon detector</td>
<td style="text-align: center; font-size: 120%; color: #31bd38">ME1/1</td>
<td style="text-align: center; font-size: 120%; color: #31bd38">ME1/2</td> 
<td style="text-align: center; font-size: 120%; color: #31bd38">ME2</td>
<td style="text-align: center; font-size: 120%; color: #31bd38">ME3</td>
<td style="text-align: center; font-size: 120%; color: #31bd38">ME4</td>
<td style="text-align: center; font-size: 120%; color: #0096ff">RE1</td>
<td style="text-align: center; font-size: 120%; color: #0096ff">RE2</td>
<td style="text-align: center; font-size: 120%; color: #0096ff">RE3</td>
<td style="text-align: center; font-size: 120%; color: #0096ff">RE4</td>
<td style="text-align: center; font-size: 120%; color: #ff3300">GE1/1</td>
<td style="text-align: center; font-size: 120%; color: #ff3300">GE2/1</td>
<td style="text-align: center; font-size: 120%; color: #ffa25f">ME0</td>
</tr>
</table>

This might look a litte bit of overkill, because initially I wanted to make use of all the possible information. It wasn't clear how useful the different muon detectors are, so I decided to use all of them. Now, I think it's clear that ME0 > CSC > GEM > iRPC > RPC, so one could try something different that uses less firmware resource.


<span style="font-size: 120%; font-weight: bold">(ii) Zone formation</span>

<span style="font-size: 120%; font-weight: bold">(iii) Patterns</span>

<span style="font-size: 120%; font-weight: bold">(iv) Pattern ghost busting</span>

### Track building

### Parameter assignment

## Performance plots

### Resolution

### Efficiency

### Rates

### Robustness

## Future plans

In [1]:
#### Please ignore

In [2]:
if True:
  from IPython.core.display import HTML
  def css_styling():
      styles = '<style>.text_cell { font-family: "Helvetica Neue","Open Sans","Liberation Sans",Arial,sans-serif; }</style>'
      return HTML(styles)
  css_styling()

In [3]:
if False:
  import graphviz as gv
  d = gv.Digraph('algorithm_blocks', engine='dot')
  d.attr(rankdir='LR')

  d.attr('node', shape='plaintext', fontname='Helvetica, sans-serif', fontsize='14')
  d.node('I', 'hits')
  d.node('O', 'tracks')

  d.attr('node', shape='square', style='rounded,filled', color='#1C7C4C', fontcolor='white', fixed_size='true', width='1.3')
  d.node('A', 'Primitive\nConversion')
  d.node('B', 'Pattern\nRecognition')
  d.node('C', 'Track\nBuilding')
  d.node('D', 'Parameter\nAssignment')

  d.attr('edge', fontname='Helvetica, sans-serif', fontsize='12', arrowsize='0.6')
  #d.edges(['IA', 'AB', 'BC', 'CD', 'DO'])
  d.edge('I', 'A', label='')
  d.edge('A', 'B', label='conv.\nhits')
  d.edge('B', 'C', label='roads')
  d.edge('C', 'D', label='proto-\ntracks')
  d.edge('D', 'O', label='')

  d.format='svg'
  d.render()
  d